# 1.进程和线程

In [12]:
#一个进程至少一个线程，线程是最小的执行单元
#fork() 在Python中创建子进程,子进程永远返回0，而父进程返回子进程的ID。这样做的理由是，一个父进程可以fork出很多子进程，
#所以，父进程要记下每个子进程的ID，而子进程只需要调用getppid()就可以拿到父进程的ID。
import os
print("Process[%s]start...." %os.getpid())
pid = os.fork()
if pid == 0:
    print("This is child process [%s] ,parent process is [%s]" %(os.getpid(),os.getppid()))
else:
    print("I [%s]  just create child process [%s]"  %(os.getpid(),pid))
      

Process[19835]start....
I [19835]  just create child process [26695]
Process[19835]start....
This is child process [26695] ,parent process is [19835]


In [24]:
# multiprocessing模块就是跨平台版本的多进程模块。
# multiprocessing模块提供了一个Process类来代表一个进程对象
from multiprocessing import Process
import os,time
def run_child(name):
    print("Child process starting...")
    print("My name is : %s, child process,pid[%s]..." %(name,os.getpid()))
    time.sleep(5)
    print("Child process end...")
print("Parent process  id : %s ..." %os.getpid())
p = Process(target=run_child,args=("test",))
print("Child process will start ...")
p.start()
#join()方法可以等待子进程结束后再继续往下运行，通常用于进程间的同步。
p.join()
print("Process run end ...")

Parent process  id : 19835 ...
Child process will start ...
Child process starting...
My name is : test, child process,pid[26885]...
Child process end...
Process run end ...


In [35]:
#如果需要启动大量子进程，可以采用进程池的方法批量创建。
import os,time,random
from multiprocessing import Pool
def long_time_task(name):
    print("run task process is : %s" %os.getpid())
    start_time = time.time()
    time.sleep(random.random() * 5)
    end_time = time.time()
    total_time =  end_time - start_time 
    print("Task %s cost : %0.2f " %(name,total_time) ) 
    print("Task: %s run end " %name)

print("Parent process is : %s" %os.getpid())
p = Pool(4)
for i in range(5):
     p.apply_async(long_time_task,args=(i,))
print("Waiting all subprocess done...")
p.close()
p.join()
print("All  subprocess done")

#对Pool对象调用join()方法会等待所有子进程执行完毕，调用join()之前必须先调用close()，调用close()之后就不能继续添加新的Process了



Parent process is : 19835
run task process is : 27887
run task process is : 27888
run task process is : 27889
run task process is : 27890
Task 2 cost : 0.07 
Task: 2 run end 
run task process is : 27889
Waiting all subprocess done...
Task 1 cost : 1.19 
Task: 1 run end 
Task 4 cost : 2.43 
Task: 4 run end 
Task 3 cost : 2.57 
Task: 3 run end 
Task 0 cost : 3.29 
Task: 0 run end 
All  subprocess done
run task process is : 19835
Task do exe cost : 0.30 
Task: do exe run end 


In [44]:
#很多时候，子进程并不是本身，而是外部进程，利用subprocess模块，可以控制输入和输出
import subprocess
print("$ nslookup www.python.org")
r = subprocess.call(['nslookup','www.python.org'])
print("Exit cod : ",r) 

$ nslookup www.python.org
Exit cod :  0


In [43]:
#如果子进程还需要输入，则可以通过communicate()方法输入：
import subprocess
print("$ nslookup")
p = subprocess.Popen(['nslookup'], stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
output, err = p.communicate(b'set q=mx\npython.org\nexit\n')
print(output.decode('utf-8'))
print('Exit code:', p.returncode)

$ nslookup
Server:		127.0.0.53
Address:	127.0.0.53#53

Non-authoritative answer:
python.org	mail exchanger = 50 mail.python.org.

Authoritative answers can be found from:


Exit code: 0


In [50]:
#进程间通信
from multiprocessing import Process, Queue
import os, time, random

def write(q):
    print("process write .... %s " %os.getpid())
    for value in ["q","w","x",1,3,3,3,3,3,3,3,3,3,3,3,]:
        print("put %s to queue" %value)
        q.put(value)
        time.sleep(random.random() *3)
def read_q(q):
    print("process read ... %s" %os.getpid())
    while True:
        value = q.get(True)
        print("get value %s from queue..." %value)

#父进程创建Queue，并传给各个子进程
q = Queue()
pw = Process(target=write,args=(q,))
pr = Process(target=read_q,args=(q,))
#启动写进程
pw.start()
#启动读进程
pr.start()
#等待pw结束
pw.join()
#强制终止pr
pr.terminate()

        

process write .... 30161 
process read ... 30162
put q to queue
get value q from queue...
put w to queue
get value w from queue...
put x to queue
get value x from queue...
put 1 to queue
get value 1 from queue...
put 3 to queue
get value 3 from queue...
put 3 to queue
get value 3 from queue...
put 3 to queue
get value 3 from queue...
put 3 to queue
get value 3 from queue...
put 3 to queue
get value 3 from queue...
put 3 to queue
get value 3 from queue...
put 3 to queue
get value 3 from queue...
put 3 to queue
get value 3 from queue...
put 3 to queue
get value 3 from queue...
put 3 to queue
get value 3 from queue...
put 3 to queue
get value 3 from queue...
